In [29]:
! pip install lyricsgenius
! pip install spotipy
! pip install pandarallel

In [55]:
import json
import lyricsgenius as lg
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pandarallel import pandarallel

In [31]:
# pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [56]:
cred = json.load(open(".\keys.json"))

In [57]:
genius = lg.Genius(cred["genius.com"]["token"], skip_non_songs=True, timeout=300, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
spotifyAuth = SpotifyClientCredentials(client_id=cred["spotify.com"]["client ID"], client_secret=cred["spotify.com"]["client secret"])
spotify = sp.Spotify(client_credentials_manager=spotifyAuth)

In [76]:
art = spotify.search(
    q="genre:rap, year:2018", 
    type="artist", 
    market="US",
    limit=10
)
print(
    json.dumps(art,
indent= 4)
)

{
    "artists": {
        "href": "https://api.spotify.com/v1/search?query=genre%3Arap%2C+year%3A2018&type=artist&market=US&offset=0&limit=10",
        "items": [
            {
                "external_urls": {
                    "spotify": "https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4"
                },
                "followers": {
                    "href": null,
                    "total": 80335798
                },
                "genres": [
                    "canadian hip hop",
                    "canadian pop",
                    "hip hop",
                    "pop rap",
                    "rap"
                ],
                "href": "https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4",
                "id": "3TVXtAsR1Inumwj472S9r4",
                "images": [
                    {
                        "height": 640,
                        "url": "https://i.scdn.co/image/ab6761610000e5eb4293385d324db8558179afd9",
                        "w

In [79]:
sample = {
    "Artist Name" : [],
    "Album" : [],
    "Track Name" : [],
    "Track Duration" : [],
    "Is_Explicit" : [],
    "Release Year" : [],
    "Lyrics" : []
    # "Popularity" : []
    
}
album_limit = 1

for artist in art["artists"]["items"]:
    name = artist["name"]
    art_id = artist["id"]
    alb = spotify.artist_albums(
        artist_id=art_id,
        album_type="album",
        country="US",
        limit=album_limit
    )
    release_year = alb["items"][0]["release_date"]
    alb_id = alb["items"][0]["id"]
    alb_name = alb["items"][0]["name"]
    tracks = spotify.album_tracks(
        album_id=alb_id
    )
    counter = 0
    artist_obj = genius.search_artist(name, max_songs=1)
    for song in tracks["items"]:
        track = song["name"]
        counter += 1
        sample["Track Name"].append(song["name"])
        sample["Is_Explicit"].append(song["explicit"])
        sample["Track Duration"].append(song["duration_ms"])
        if "feat" in track:
            s = artist_obj.song(track[:track.index("feat")-2])
        else:
            s = artist_obj.song(track)
        
        sample["Lyrics"].append("\n".join(s.lyrics.split("\n")[1:]) if s is not None else "Not Found")
        
    sample["Album"] += [alb_name] * counter
    sample["Artist Name"] += [name] * counter
    sample["Release Year"] += [release_year] * counter
    



Searching for songs by Drake...

Song 1: "God’s Plan"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Rich Flex" by Drake...
Done.
Searching for "Major Distribution" by Drake...
Done.
Searching for "On BS" by Drake...
Done.
Searching for "BackOutsideBoyz" by Drake...
Done.
Searching for "Privileged Rappers" by Drake...
Done.
Searching for "Spin Bout U" by Drake...
Done.
Searching for "Hours In Silence" by Drake...
Done.
Searching for "Treacherous Twins" by Drake...
Done.
Searching for "Circo Loco" by Drake...
Done.
Searching for "Pussy & Millions" by Drake...
Done.
Searching for "Broke Boys" by Drake...
Done.
Searching for "Middle of the Ocean" by Drake...
Done.
Searching for "Jumbotron Shit Poppin" by Drake...
Done.
Searching for "More M’s" by Drake...
Done.
Searching for "3AM on Glenwood" by Drake...
No results found for: '3AM on Glenwood Drake'
Searching for "I Guess It’s Fuck Me" by Drake...
Done.
Searching for songs by Travis Scott...

Song 1: "SICKO MO

In [80]:
df = pd.DataFrame(sample)
df

,Artist Name,Album,Track Name,Track Duration,Is_Explicit,Release Year,Lyrics
0,Drake,Her Loss,Rich Flex,239359,True,2022-11-04,"\nGo buy a zip of weed, hit the club\nPay for ..."
1,Drake,Her Loss,Major Distribution,170501,True,2022-11-04,"No, no, no, no, no, no\nNo, no, no, no, no\nMa..."
2,Drake,Her Loss,On BS,261543,True,2022-11-04,"Yeah, woah, woah, woah\n\nI had to cut some ni..."
3,Drake,Her Loss,BackOutsideBoyz,152961,True,2022-11-04,"BackOutsideBoyz, totin' a seventy on the strip..."
4,Drake,Her Loss,Privileged Rappers,160821,True,2022-11-04,"Woah, woah, yeah\nLook at me dead in my eyes, ..."
...,...,...,...,...,...,...,...
190,Kendrick Lamar,Mr. Morale & The Big Steppers,Auntie Diaries,281069,True,2022-05-13,Heart plays in ways the mind can't figure out\...
191,Kendrick Lamar,Mr. Morale & The Big Steppers,Mr. Morale,210841,True,2022-05-13,It was one of the worst performances I've seen...
192,Kendrick Lamar,Mr. Morale & The Big Steppers,Mother I Sober (feat. Beth Gibbons of Portishead),406528,True,2022-05-13,"I'm sensitive, I feel everything, I feel every..."
193,Kendrick Lamar,Mr. Morale & The Big Steppers,Mirror,256006,True,2022-05-13,"I choose me\n\nThe pressure's taking over me, ..."


In [81]:
df.to_csv(".\Sample.csv")

In [82]:
df.head()

,Artist Name,Album,Track Name,Track Duration,Is_Explicit,Release Year,Lyrics
0,Drake,Her Loss,Rich Flex,239359,True,2022-11-04,"\nGo buy a zip of weed, hit the club\nPay for ..."
1,Drake,Her Loss,Major Distribution,170501,True,2022-11-04,"No, no, no, no, no, no\nNo, no, no, no, no\nMa..."
2,Drake,Her Loss,On BS,261543,True,2022-11-04,"Yeah, woah, woah, woah\n\nI had to cut some ni..."
3,Drake,Her Loss,BackOutsideBoyz,152961,True,2022-11-04,"BackOutsideBoyz, totin' a seventy on the strip..."
4,Drake,Her Loss,Privileged Rappers,160821,True,2022-11-04,"Woah, woah, yeah\nLook at me dead in my eyes, ..."


In [98]:
avg_duration = df.groupby("Artist Name").mean()["Track Duration"].to_frame().reset_index()
avg_duration["avg"] = avg_duration.apply(lambda x: f"{x['Track Duration']/60000 :.2f} min {x['Track Duration']%60000} seconds", axis=1)#print(f"{x['Artist Name']} : {x['Track Duration']/60000 :.2f} min {x['Track Duration']%60000} seconds"), axis=1)

C:\Users\aanan\AppData\Local\Temp\ipykernel_13680\3279372236.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_duration = df.groupby("Artist Name").mean()["Track Duration"].to_frame().reset_index()


In [99]:
avg_duration

,Artist Name,Track Duration,avg
0,$uicideboy$,173056.000000,2.88 min 53056.0 seconds
1,21 Savage,227534.062500,3.79 min 47534.0625 seconds
2,Drake,227534.062500,3.79 min 47534.0625 seconds
3,Future,178764.409091,2.98 min 58764.40909090909 seconds
4,Kanye West,245788.625000,4.10 min 5788.625 seconds
5,Kendrick Lamar,248698.210526,4.14 min 8698.210526315786 seconds
6,Lil Baby,170452.869565,2.84 min 50452.86956521738 seconds
7,Post Malone,188596.555556,3.14 min 8596.555555555562 seconds
8,SZA,177578.434783,2.96 min 57578.4347826087 seconds
9,Travis Scott,232462.684211,3.87 min 52462.68421052632 seconds


In [ ]:
avg_duration[["Artist Name", "Track Duration"]]